In [3]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [7]:

client = RESTClient(api_key=token)

In [8]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



393359


In [9]:
aggs[-1]

Agg(open=192.1505, high=192.1505, low=192.1501, close=192.1501, volume=2147, vwap=192.1512, timestamp=1703897940000, transactions=12, otc=None)

In [10]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [11]:
from heston_param import *

In [12]:
aapl_aggs.groupby("day")

In [13]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [14]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [1]:
import pickle
from heston_param import *
hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [2]:
print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 50))

tensor([ 0.1798,  0.1025,  0.7553, -0.0219,  0.4685], dtype=torch.float64)


In [3]:
while True:
	try:
		print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 50))
		break
	except RuntimeError:
		continue

tensor([3.4741, 0.2184, 0.6046, 0.8979, 0.2477], dtype=torch.float64)


In [ ]:
def single_day_calibration(args):
		i, day = args
		while True:
			try:
				params = calibrate_daily_parameters(hist_volatilities[i], 0.1, day["close"].values, 0.0237, day["close"].values.shape[0], 300)
				
				return i, params
			except RuntimeError:
				continue

In [8]:
len(daily_aggs)

1273

In [6]:
daily_params = np.empty((len(daily_aggs), 5))
for i, day in enumerate((daily_aggs[:20])):
	while True:
		try:
			daily_params[i] = calibrate_daily_parameters(hist_voilatilities[i], 0.1, daily_aggs[i]["close"].values, 0.0237, daily_aggs[i]["close"].values.shape[0], 50)
			if (i + 1) % 10 == 0:
					rate =  100 *  np.round((i + 1) /day["close"].values.shape[0], 2)
					print(f"{rate}% completed.")
			
			break
		except RuntimeError:
			continue

daily_params

1.82% completed.
2.42% completed.


array([[ 7.97825366e+000,  2.72294350e-002,  9.72123831e-001,
         5.28063392e-001,  8.81649295e-001],
       [ 4.26833676e+000,  4.63777140e-001,  9.20422583e-001,
        -3.24032695e-002,  3.25410263e-002],
       [ 2.76040058e+000,  3.63141695e-001,  3.23396977e-001,
        -8.56510357e-001,  2.62603938e-001],
       ...,
       [ 9.72245510e-067,  4.42132206e-062,  1.05249949e-153,
         3.86045603e-224,  5.58166871e-091],
       [ 9.72245510e-067,  4.42132206e-062,  1.05249949e-153,
         3.86046015e-224,  5.58166871e-091],
       [ 9.72245510e-067,  4.42132206e-062,  1.05249949e-153,
         3.86046427e-224,  5.58166871e-091]])